In [32]:
"""    
    Indicador: Porcentaje de Hectáreas actualizadas con respecto al total de hectáreas que deben actualizarse para cumplir un Hito contractual.
"""

'    \n    Indicador: Porcentaje de Hectáreas actualizadas con respecto al total de hectáreas que deben actualizarse para cumplir un Hito contractual.\n'

<h3>Importación Librerías</h3>

In [33]:
import pandas as pd
import os
import seaborn as sns

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos
import funcion_SQL_a_DataFrame

<h3>Parametrización de Variables</h3>

In [34]:
consulta_tRadicados = """ 
    select t1.municipio
        ,t1.npn
        ,t1.nro_ficha
        ,t1.radicado
        ,(case when t1.vr_nombre_proceso like 'Comple%%' then 'Actualización'
            when t1.vr_nombre_proceso like 'Forma%%' then 'Actualización'
            when t1.vr_nombre_proceso like 'Muta%%' then 'Conservación'				
            when t1.vr_nombre_proceso like 'Recti%%' then 'Conservación'
            else t1.vr_nombre_proceso
            end) "Proceso"
        ,t1.vr_nombre_proceso	
        ,t1.estado_tramite
        ,t1.actividad
        ,t1.vr_estado_paso
        ,t2.vr_proegn_est proceso_estado
        ,t2.nb_step_idn
        ,cast(t2.dt_adt_fchupd as date) dt_adt_fchupd
        ,t2.vr_proegn_ori
        ,cast(t1.fecha_radicado as date) fecha_radicado
        ,t1.usuario
        ,cast(t2.dt_adt_fchupd as date) fecha_actualizacion_tramite
        ,t3.vr_proces_des
        ,t6.vr_consec_nom
        ,t6.vr_consec_des
        ,t6.vr_consec_tip
        ,t4.vr_step_est
        --,t4.nb_task_idn
        ,t5.vr_task_nom 
        ,t5.vr_task_tip
        ,t5.vr_task_con
        ,t5.vr_task_est 
        ,t7.vr_functy_des
    from vw_rpt_tramites_en_proceso t1 
    left join tb_bpm_egn_proegn t2 on t1.radicado = t2.vr_proegn_rad
    left join tb_bpm_mdl_proces t3 on t2.nb_proces_idn = t3.nb_proces_idn 
    left join tb_fw_sec_consec t6 on t3.nb_consec_idn = t6.nb_consec_idn 
    left join tb_bpm_egn_step t4 on t2.nb_step_idn = t4.nb_step_idn
    left join tb_bpm_mdl_task t5 on t4.nb_task_idn = t5.nb_task_idn
    left join tb_fw_gui_functy t7 on t5.nb_functy_idn = t7.nb_functy_idn;
"""

consuta_tFinalizados = """
    select t1.municipio
        ,t1.npn
        ,t1.nro_ficha
        ,t1.radicado
        ,(case when t1.vr_nombre_proceso like 'Comple%%' then 'Actualización'
            when t1.vr_nombre_proceso like 'Forma%%' then 'Actualización'
            when t1.vr_nombre_proceso like 'Muta%%' then 'Conservación'				
            when t1.vr_nombre_proceso like 'Recti%%' then 'Conservación'
            else t1.vr_nombre_proceso
            end) "Proceso"
        ,t1.vr_nombre_proceso
        ,cast(t1.fecha_radicado as date) fecha_radicado
        ,cast(t1.fecha_finalizacion as date) fecha_finalizacion
        ,t1.resolucion
        ,t1.fecha_resolucion
        --,t1.nb_proegn_idn
        ,t2.vr_proegn_est proceso_estado
        --,t2.nb_step_idn
        --,cast(t2.dt_adt_fchupd as date) dt_adt_fchupd
        ,t2.vr_proegn_ori
        --,t2.nb_proces_idn
        --,t3.vr_proces_nom 
        ,t3.vr_proces_des
        ,t6.vr_consec_nom
        ,t6.vr_consec_des
        ,t6.vr_consec_tip
        ,t4.vr_step_est
        --,t4.nb_task_idn
        ,t5.vr_task_nom 
        ,t5.vr_task_tip
        ,t5.vr_task_con
        ,t5.vr_task_est 
        ,t7.vr_functy_des	
    from bpmcat.vw_rpt_tramites_finalizados t1
    left join tb_bpm_egn_proegn t2 on t1.nb_proegn_idn = t2.nb_proegn_idn
    left join tb_bpm_mdl_proces t3 on t2.nb_proces_idn = t3.nb_proces_idn 
    left join tb_fw_sec_consec t6 on t3.nb_consec_idn = t6.nb_consec_idn 
    left join tb_bpm_egn_step t4 on t2.nb_step_idn = t4.nb_step_idn
    left join tb_bpm_mdl_task t5 on t4.nb_task_idn = t5.nb_task_idn
    left join tb_fw_gui_functy t7 on t5.nb_functy_idn = t7.nb_functy_idn;
"""

columnasRadicadoActualizacionCampo = ['municipio','npn','nro_ficha','radicado','estado_tramite','actividad','usuario','vr_proces_des']


DIRECTORIO_REPORTE = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuirica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\zReportes"
NOMBRE_REPORTE_RADSINNPN = 'Reporte_Radicado_SinNPNoFicha.xlsx'
NOMBRE_REPORTE_RADFORMACIONATENDIENDO = 'Reporte_Radicado_ActualizacionEnAtencion.xlsx'

RUTA_BD_LOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuirica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\_2_2_0.gdb"
RUTA_BD_CONSOLIDADA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
NOMBRE_CAPA_HAXESTADO = 'TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS'

<h3>Ejecución de Funciones</h3>

In [35]:
df_consulta_radicados = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRadicados)
df_terrenos = funcion_dataframe_terrenos.dataframe_terrenos()

Inicio de la conexión
Concluye el proceso


<h3>Proceso: Actualizacion, Sin NPN Registrados</h3>

In [36]:

# TODO: Radicados sin NPN

# ** 1. Filtro
df_radicadosActualizacion_sin_npn = df_consulta_radicados[(df_consulta_radicados['Proceso'] == 'Actualización') & (df_consulta_radicados['npn'].isnull())]

total_radicadosActualizacionSinNPN = df_radicadosActualizacion_sin_npn.shape[0]
print(f"El total de radicados sin parametrización de NPN o Ficha es: {total_radicadosActualizacionSinNPN}")

df_radicadosActualizacionSinNPNParaReporte = df_radicadosActualizacion_sin_npn[['municipio','radicado','usuario','vr_consec_nom']]
df_radicadosActualizacionSinNPNParaReporte.to_excel(os.path.join(DIRECTORIO_REPORTE, NOMBRE_REPORTE_RADSINNPN), sheet_name='ReporteRadSinNPN')
print(f"Se crea el reporte {NOMBRE_REPORTE_RADSINNPN}")

df_radicadosActualizacionSinNPN = pd.DataFrame([[total_radicadosActualizacionSinNPN]], columns=['TotalRadicadosSinNPN'])

El total de radicados sin parametrización de NPN o Ficha es: 14
Se crea el reporte Reporte_Radicado_SinNPNoFicha.xlsx


<h3>Proceso: Actualizacion - Formación y Actualización - Atendiendo</h3>

In [37]:
df_radicadosActualizacionConNPNAtendiendo = df_consulta_radicados[(df_consulta_radicados['Proceso'] == 'Actualización') & 
    (df_consulta_radicados['npn'].notnull()) & 
    (df_consulta_radicados['vr_estado_paso'] == 'ATENDIENDO') &
    (df_consulta_radicados['vr_nombre_proceso'].str.startswith('Form'))]

df_radicadosActualizacionConNPNAtendiendoCampo = df_radicadosActualizacionConNPNAtendiendo[columnasRadicadoActualizacionCampo]

df_radicadosActualizacionConNPNAtendiendoCampo.to_excel(os.path.join(DIRECTORIO_REPORTE, NOMBRE_REPORTE_RADFORMACIONATENDIENDO), sheet_name='ReporteActualizacionEnAtencion')
print(f"Se crea el reporte {NOMBRE_REPORTE_RADFORMACIONATENDIENDO}")

Se crea el reporte Reporte_Radicado_ActualizacionEnAtencion.xlsx


<h3>Proceso: Actualizacion - Formación y Actualización - Atendiendo (ESPACIALIZACIÓN)</h3>

In [38]:
df_terrenos_radicadosActualizacionConNPNAtendiendoCampo = pd.merge(left=df_terrenos, right=df_radicadosActualizacionConNPNAtendiendoCampo, left_on="CODIGO", right_on="npn", how="inner")
df_radicadosActualizacionConNPNAtendiendoCampo = df_terrenos_radicadosActualizacionConNPNAtendiendoCampo[['Area_ha_cmt12','estado_tramite']]

# TODO: Hectareaje por Tramite de los Radicados En Atención por parte de Reconocimiento
df_HectareajeXPorEstadoTramite = pd.DataFrame(df_radicadosActualizacionConNPNAtendiendoCampo.groupby(by='estado_tramite')['Area_ha_cmt12'].sum())

# TODO: ExportacionTramitesEnAtencionXEstado

# ** Exportación a BD Consolidada
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{NOMBRE_CAPA_HAXESTADO.upper()}"
ruta_salida_terrenos_ha_x_estado = os.path.join(RUTA_BD_CONSOLIDADA, NOMBRE_CAPA_SALIDA)

df_terrenos_radicadosActualizacionConNPNAtendiendoCampo.spatial.to_featureclass(location=ruta_salida_terrenos_ha_x_estado)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA} a BD Consolidada")

# ** Exportación a BD Seguimiento Indicadores
df_terrenos_radicadosActualizacionConNPNAtendiendoCampo.spatial.to_featureclass(location=os.path.join(RUTA_BD_LOCAL,NOMBRE_CAPA_HAXESTADO))
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA} a BD Local 2.2.0")

Se exporta la capa Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS a BD Consolidada
Se exporta la capa Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS a BD Local 2.2.0
